NaiveBayes Class


In [12]:
class NaiveBayes:
    wordCounts = {}
    def __init__(self):
        self.wordCounts = {}

    def add_to_word_count(self,text,label):
        listOfWords = BagOfWords(text)
        if self.wordCounts.get(label) == None:
            self.wordCounts[label] = listOfWords
        else:
            for word in listOfWords:
                if self.wordCounts[label].get(word) == None:
                    self.wordCounts[label][word] = listOfWords[word]
                else:
                    self.wordCounts[label][word] += listOfWords[word]
        return
    
    def add_word_threshold(self, threshold):
        items_below_threshold = []
        for label in self.wordCounts.keys():
            for word in self.wordCounts[label]:
                if self.wordCounts[label][word] <= threshold:
                     items_below_threshold.append([label, word])
        
                     
        for item in items_below_threshold:    
            del self.wordCounts[item[0]][item[1]]

    def predict_text(self,text):
        wordarr = BagOfWords(text)
        labels = list(self.wordCounts.keys())
        probabilities = {}
        
        for label in labels:
            probabilities[label] = 1
            
        #find highest probility
        for word in wordarr:
            for label in labels:
                prob = self.calculate_probability(word,label)
                if prob > 0:
                    probabilities[label] *= prob
        
        highest = probabilities[labels[0]]
        highestlabel = labels[0]
        for label in labels:
            #print(probabilities[label])
            if probabilities[label] > highest:
                #print("prev highest", highestlabel,'new:',label)
                highest = probabilities[label]
                highestlabel = label
     
        return highestlabel
        
    
    def calculate_probability(self,word,lab):
        labels = self.wordCounts.keys()
        total = 0
        labcount = self.wordCounts[lab].get(word,0)
        for label in labels:
            total += self.wordCounts[label].get(word,0)
        #print(word," ",lab,":",labcount,total,labcount/total )
        if (labcount == 0 or total == 0):
            return 0
        else:
            return labcount/total

def BagOfWords(wordarr):
    wordCount = {}
    
    for word in wordarr:
            if wordCount.get(word) != None:
                wordCount[word] += 1
            else:
                wordCount[word] = 1
    return wordCount

Data is a set of articles that are labeled either objective or subjective
Data: https://archive.ics.uci.edu/ml/machine-learning-databases/00450/
Data/
    Features.csv
    Text0001.txt
    Text0001.txt
    ............
    Text1000.txt
    
Features.csv: Headers for the data
Text0001,...,objective,....
Text0700,...,subjective,....

TextXXXX.txt: Article full of words

In [4]:
with open("Data/features.csv") as file:
    head = [next(file) for x in range(2)]
print(head)


['TextID,URL,Label,totalWordsCount,semanticobjscore,semanticsubjscore,CC,CD,DT,EX,FW,INs,JJ,JJR,JJS,LS,MD,NN,NNP,NNPS,NNS,PDT,POS,PRP,PRP$,RB,RBR,RBS,RP,SYM,TOs,UH,VB,VBD,VBG,VBN,VBP,VBZ,WDT,WP,WP$,WRB,baseform,Quotes,questionmarks,exclamationmarks,fullstops,commas,semicolon,colon,ellipsis,pronouns1st,pronouns2nd,pronouns3rd,compsupadjadv,past,imperative,present3rd,present1st2nd,sentence1st,sentencelast,txtcomplexity\n', 'Text0001,http://msn.foxsports.com/foxsoccer/mexico/story/mexican-review-jan-19-toluca-secures-first-victory-beats-leon-011913,objective,109,0,1,7,9,0,5,8,6,0,0,0,0,29,0,0,12,0,0,1,2,2,0,0,2,0,3,0,0,11,0,2,0,0,0,1,0,1,0,2,0,0,0,4,2,0,0,0,0,0,3,0,11,0,0,0,0,1,18\n']


In [11]:
with open('Data/Text0001.txt') as f:
    print(f.read())

Finalists in the Apertura play-offs, Toluca had drawn their first two Clausura games but got off to a good start when Edgar Benitez put them ahead in the 16th minute.
Matias Britos levelled 20 minutes later but Lucas Silva netted 14 minutes from the end to ensure the visitors took all three points.
  	Franco Arizala scored 13 minutes from time to ensure Jaguares claimed their first point with a 1-1 draw against Monterrey, who had opened the scoring through Aldo De Nigris (14).
 Hosts Jaguares also had Jorge Rodriguez sent off in the closing moments.


Time to parse and celanup the data. Since the words and labels are not in the same documet, this becomes slightly more complex. 

In [13]:
import csv

def ParseFeatures(file_path):
    features = {}
    with open(file_path) as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        next(reader)
        for row in reader:
            features[row[0]] = Article(0 if row[2] == "objective" else 1)
    return features

def GetWordsFromArticle(path):
    with open(path, encoding="ISO-8859-1") as f:
        words = []
        for row in f.read().split():
            # Remove all special characters
            word = ''.join(e for e in row if e.isalnum())
            words.append(word)
    return words

class Article:
    def __init__(self, label):
         self.label = label
         self.words = []

Time to start using the data

In [24]:
data = ParseFeatures("Data/features.csv")
print(data["Text0001"].label, end=' ')
print(data["Text0001"].words)

0 []


In [25]:
for key in data:
    data[key].words = GetWordsFromArticle("Data/" + key + ".txt")
print(data["Text0001"].label,end=' ')
print(data["Text0001"].words)

0 ['Finalists', 'in', 'the', 'Apertura', 'playoffs', 'Toluca', 'had', 'drawn', 'their', 'first', 'two', 'Clausura', 'games', 'but', 'got', 'off', 'to', 'a', 'good', 'start', 'when', 'Edgar', 'Benitez', 'put', 'them', 'ahead', 'in', 'the', '16th', 'minute', 'Matias', 'Britos', 'levelled', '20', 'minutes', 'later', 'but', 'Lucas', 'Silva', 'netted', '14', 'minutes', 'from', 'the', 'end', 'to', 'ensure', 'the', 'visitors', 'took', 'all', 'three', 'points', 'Franco', 'Arizala', 'scored', '13', 'minutes', 'from', 'time', 'to', 'ensure', 'Jaguares', 'claimed', 'their', 'first', 'point', 'with', 'a', '11', 'draw', 'against', 'Monterrey', 'who', 'had', 'opened', 'the', 'scoring', 'through', 'Aldo', 'De', 'Nigris', '14', 'Hosts', 'Jaguares', 'also', 'had', 'Jorge', 'Rodriguez', 'sent', 'off', 'in', 'the', 'closing', 'moments']



Setup our training and validation. First we need to create a class

In [31]:
import random
class CrossValidationManager:
    k = 0
    data_length = 0
    data_indexes = {}
    current_index = -1

    def __init__(self, k_times, data, seed = 0):
        self.k = k_times
        self.data = data
        if (seed > 0):
            random.Random(seed).shuffle(self.data)
        else:
            random.shuffle(self.data)
        self._get_indexes()
    
    def _get_indexes(self):
        length = int(len(self.data) / self.k)
        remainder = len(self.data) % self.k

        if remainder > 0:
            length += 1
        else:
            remainder = -1

        for index in range(self.k):
            if remainder > 0:
                remainder -= 1
            elif remainder == 0:
                length -= 1
                remainder = -1

            if index == 0:
                self.data_indexes[index] = [0,length]
            else:
                previous_end_index = self.data_indexes[index-1][1]
                self.data_indexes[index] = [previous_end_index, previous_end_index + length]
    
    def data_available(self):
        self.current_index += 1
        if self.current_index == self.k:
            return False
        return True

    def get_training_data(self):
        data = []
        for i in range(self.k):
            if i == self.current_index:
                continue
            s = self.data_indexes[i][0]
            e = self.data_indexes[i][1]
            data += self.data[s:e]
        return data
    
    def get_validation_data(self):
        s = self.data_indexes[self.current_index][0]
        e = self.data_indexes[self.current_index][1]
        return self.data[s:e]

In [32]:


K_FOLD_COUNT = 5
# Changing the seed will shuffle the data differently but consistently between runs
# 0 will shuffle differently every run of the program
SEED = 12

dataManager = CrossValidationManager(K_FOLD_COUNT, list(data), seed=SEED)



We have all of our data setup. Send it to our alg and make some predictions

In [33]:
while dataManager.data_available():
    n = NaiveBayes()
    trainData = dataManager.get_training_data()
    valData = dataManager.get_validation_data()

    # Train
    for article_name in trainData:
        n.add_to_word_count(GetWordsFromArticle("Data/" + article_name + ".txt"), data[article_name].label)
    
    #n.add_word_threshold(2)

    #Test
    validationData = {}
    for article_name in valData:
        validationData[article_name] = [GetWordsFromArticle("Data/" + article_name + ".txt"), data[article_name].label]

    correct = 0
    for words in validationData.items():
        prediction = n.predict_text(words[1][0])
        
        if prediction == words[1][1]:
            correct += 1
    print((correct / len(validationData)) * 100, end='')
    print(" Correct: " + str(correct) + "/" + str(len(validationData)))

71.0 Correct: 142/200
60.5 Correct: 121/200
79.5 Correct: 159/200
82.0 Correct: 164/200
76.5 Correct: 153/200
